In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

import json


In [ ]:
# Read in ratings data
def loadMovieRatings(filename,path="ml-latest-small/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (user,movieid,rating,ts)=line.split(',')
            if (user != "userId"):
                data.append({ "user_id": str(user), "movie_id": str(movieid)})
                y.append(float(rating))
                users.add(user)
                items.add(movieid)

    return (data, np.array(y), users, items)


In [ ]:
# Read in movie ID and movie names
def loadMovieNames(filename,items,path="ml-latest-small/"):
    movie_id_name = []
    words = []
    #filename="u.item"
    with open(path+filename,encoding='latin-1') as f:
        for line in f:
            words = line.split(',')
            movie_id = words[0]
            movie_tmdbid = words[1]
            movie_poster = words[2]
            movie_genre = words[3]
            movie_name = words[4]
            movie_name = movie_name.rstrip()
            #movie_id_name.insert(movie_id,movie_name)
            #(movie_id,movie_name,other_stuff)=line.split('|')
            if (movie_id != "movieId"):    
                movie_id_name.append({ "movie_name": str(movie_name), "movie_id": str(movie_id), "movie_genre": str(movie_genre), "movie_tmdbid": str(movie_tmdbid), "movie_poster": str(movie_poster)})
            #y.append(float(rating))
            #users.add(user)
            #items.add(movieid)
        #print(movie_id_name)

    return (movie_id_name)


In [ ]:
# Read in movie ID and return TMDBID for URL and Posters
def getTmdbId(filename,items,path="ml-latest-small/"):
    words = []
    with open(path+filename,encoding='utf-8') as f:
        for line in f:
            words = line.split(',')
            movie_id = words[0]
            tmdbid = words[2]
            #movie_id_name.insert(movie_id,movie_name)
            #(movie_id,movie_name,other_stuff)=line.split('|')
            if (items == movie_id):
                #print(movie_id)
                return tmdbid

    return (0)


In [ ]:
# Read in movie ID and return Movie Name
def getMovieName(movie_id,movie_id_name):
    for movie in movie_id_name:
        if (movie['movie_id'] == movie_id):
            #print("Inside Function "+movie_id)
            return(movie['movie_name'])
        
    

In [ ]:
# Read in movie ID and return Movie URL
def getMovieGenre(movie_id,movie_id_name):
    for movie in movie_id_name:
        if (movie['movie_id'] == movie_id):
            #print("Inside Function "+movie_id)
            return(movie['movie_genre'])
        

In [ ]:
# Read in movie ID and return Movie Poster
def getMoviePoster(movie_id,movie_id_name):
    for movie in movie_id_name:
        if (movie['movie_id'] == movie_id):
            #print("Inside Function "+movie_id)
            return(movie['movie_poster'])

In [ ]:
# Read in movie ID and return Movie URL
def getMovieTmdbId(movie_id,movie_id_name):
    for movie in movie_id_name:
        if (movie['movie_id'] == movie_id):
            #print("Inside Function "+movie_id)
            return(movie['movie_tmdbid'])
        

In [ ]:
(train_data, y_train, train_users, train_items) = loadMovieRatings("ratings.csv")
#(test_data, y_test, test_users, test_items) = loadMovieRatings("ua.test")
(movie_id_name) = loadMovieNames("movies.csv",train_items)
v = DictVectorizer()
X_train = v.fit_transform(train_data)
#X_test = v.transform(test_data)

In [ ]:
train_data

In [ ]:
for movie in movie_id_name:
    print(movie['movie_id'])
    print(movie['movie_name'])
    print(movie['movie_genre'])
    print(movie['movie_tmdbid'])
    print(movie['movie_poster'])

In [ ]:
print(X_train)

In [ ]:
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

In [ ]:
fm.fit(X_train,y_train)

In [ ]:
preds = fm.predict(X_test)

In [ ]:
fm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
app_df = pd.DataFrame(columns=['user_id','movie_id','rating'])

In [ ]:
for user in list(train_users):
    print(user)

In [ ]:
try:
    f = open("predicted_ratings_final.csv", "w")
    f.write("userId\t"+"movieId\t"+"rating\t"+"tmdbId\t"+"moviePoster\t"+"movieGenre\t"+"movieName")
    f.write("\n")
except:
    print("file open error")
    
for user in list(train_users):
    #print(user)
    if (int(user) < 10):
        for item in list(train_items):
            new_predict = fm.predict(v.transform([{'movie_id': str(item), 'user_id': str(user)}]))
            try:
                #print(user)
                movie_name = getMovieName(item,movie_id_name)
                movie_genre = getMovieGenre(item,movie_id_name)
                movie_poster = getMoviePoster(item,movie_id_name)
                movie_genre = movie_genre.rstrip()
                tmdbid = getMovieTmdbId(item,movie_id_name)
                tmdbid = tmdbid.rstrip()
                #if (movie_name != "" & movie_genre != ""):  
                f.write(str(user)+"\t")
                f.write(str(item)+"\t")
                f.write(str(new_predict[0])+"\t")
                f.write(str(tmdbid)+"\t")
                f.write(str(movie_poster)+"\t")
                f.write(str(movie_genre)+"\t")
                f.write(str(movie_name))
                f.write("\n")
            except:
                f.write("\n")
                print("Error")
            

In [ ]:
f1